In [ ]:
import re
import os
import datetime
from datetime import datetime
import seaborn as sns
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing 
import sys
sys.tracebacklimit = 2

In [ ]:
workdir = os.getcwd()
average_travel_file = "\data\Average Travel time by departure time.xlsx"
path = os.path.join(workdir+average_travel_file)

avg_commute = pd.read_excel(path,header = 4)
avg_commute.fillna(0)

def clean_locations(string):
    return re.findall("(?<=--)(.*)(?= PUMA)",string)[0]

avg_commute['Selected Geographies'] = avg_commute['Selected Geographies'].apply(lambda x: clean_locations(x))

drop_col = avg_commute.columns[1]

avg_commute.drop([drop_col],axis=1,inplace=True)
avg_commute.fillna(np.NaN,inplace=True)
avg_commute

In [ ]:
splits = []
for city in avg_commute['Selected Geographies']:
    if re.match("\w", city):
        sublist = re.split('[^\s|\w]',city)
        for item in sublist:
            if item != None:
                if 'Cities' in item:
                    item = item.strip('Cities')
                    splits.append(item)
                if "Region" in item:
                    item = item.strip('Region')
                    splits.append(item)
                else:
                    splits.append(item)
    else:
        splits.append(city)    


semi_cleaned = list(set(splits))
semi_cleaned.remove('')
semi_cleaned.remove('North')
semi_cleaned.remove('Northeast')
semi_cleaned.remove('East')
semi_cleaned.remove('Southeast')
semi_cleaned.remove('South')
semi_cleaned.remove('Southwest')
#semi_cleaned.remove('West')
semi_cleaned.remove('Northwest')
semi_cleaned.remove("Central")

cleaned = [x.strip() for x in semi_cleaned]

df = pd.DataFrame(index = avg_commute.columns[1:])
for city in cleaned:
    for geo in avg_commute['Selected Geographies']:
        if city in geo:
            df[city] = avg_commute[avg_commute['Selected Geographies'] == geo].values[:,1:].T
        
commute_by_city = df.transpose()
commute_by_city

In [ ]:
def clean_time(string):
    temp = re.findall("[\d]{1,2}:[\d]{2}[\s][\w][.][\w][.]",string)[0]
    time = temp.split(" ")
    time[1] = time[1].upper().replace(".","")
    stringy = " ".join(time)
    return stringy

commute_by_city

old_cols = list(commute_by_city.columns.values)
new_cols = pd.Series(old_cols).apply(lambda x: clean_time(x))
new_cols = pd.to_datetime(new_cols, format="%I:%M %p").dt.strftime('%H:%M')
commute_by_city.set_axis(new_cols,axis=1,inplace= True)

avg_T = commute_by_city.transpose()
avg_T.index = pd.to_datetime(avg_T.index)
avg_T = avg_T.resample('10T').sum()
avg_T.index = pd.to_datetime(avg_T.index).strftime('%H:%M')
avg_T

In [ ]:
morning = avg_T.loc["01:30":"12:00"]
morning

In [ ]:
plt.scatter(morning.index,morning['Castaic'])
fit1 = SimpleExpSmoothing(morning['Castaic'], initialization_method="heuristic").fit(smoothing_level=0.3, optimized=False)
plt.scatter(morning.index,fit1.fittedvalues)
plt.ylabel('Average Commute Time')
plt.xlabel('Time of Day')
plt.plot()
fit1.sse

In [ ]:
best_smoothing = pd.DataFrame(columns = morning.columns)
for i in range(9):
    smooth = 0.1 + i/10
    fit = morning.apply(lambda x: SimpleExpSmoothing(x, initialization_method="heuristic").fit(smoothing_level=smooth, optimized=False).sse)
    best_smoothing = best_smoothing.append(fit, ignore_index=True)
best_smoothing

In [ ]:
plt.plot(best_smoothing.apply(lambda x: np.mean(x),axis=1))
plt.title('Average SSE over entire dataset for each 0.1 increase in smoothing parameter')
plt.xlabel('smoothing coefficient')
plt.ylabel('SSE')
plt.plot()

In [ ]:
plt.scatter(morning.index,morning['Malibu'])
fit3 = SimpleExpSmoothing(morning['Malibu'], initialization_method="heuristic").fit(smoothing_level=0.5, optimized=False)
plt.scatter(morning.index,fit3.fittedvalues)
plt.title('Smoothed commute times[0.4] - Malibu')
plt.ylabel('Average Commute Time - Minutes')
plt.xlabel('Time of day')
plt.plot()
fit3.sse

In [ ]:
smoothed_commutes  = morning.apply(lambda x: SimpleExpSmoothing(x, initialization_method="heuristic").fit(smoothing_level=0.5, optimized=False).fittedvalues)
smoothed_commutes

In [ ]:
import geopy
from geopy.geocoders import GeoNames
gn = GeoNames(username='matthew.rand')

In [ ]:
commute_statistics = pd.DataFrame(index=smoothed_commutes.columns)
commute_statistics['Average Commute'] = smoothed_commutes.apply(lambda x: np.mean(x))
commute_statistics['Variation in Commute Time'] = smoothed_commutes.apply(lambda x: np.std(x))
#commute_statistics['Centroid'] = commute_statistics.apply(lambda x: gn.geocode("{} CA USA".format(x))[1][::-1]  )

commute_statistics

In [ ]:
def get_lat_long(city_name):
    locale = "{} California USA".format(city_name)
    place = gn.geocode(locale)
    if place != None:
        return place[1][::-1]
    else:
        return None

centroids = []
for index in commute_statistics.index.to_list():
    centroids.append( get_lat_long(index))

commute_statistics['Centroids'] = centroids
commute_statistics